In [1]:
import gc
import os
from datetime import datetime
import pickle
import psutil
from tqdm import tqdm, tqdm_notebook
from functools import partial, update_wrapper
from itertools import product
from collections import OrderedDict
from multiprocessing import Pool

import bloscpack as bp

import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression

from YSMLT import utils as g_utils
from YSMLT.series import utils as ts_utils

from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy.signal import welch, find_peaks
from scipy import stats
from scipy.special import entr
from scipy.stats import entropy
from tsfresh.feature_extraction import feature_calculators

In [7]:
feats_f_curr = [f for f in os.listdir('../input/feats_tblr') if ('trn_feat' in f) and ('w250' in f)][0]
feats_f_curr

'trn_feat_g4_w250.bp'

In [8]:
feats_f_trgt = [f for f in os.listdir('../input/feats_tblr') if ('tst_feat' in f) and ('w250' in f)][0]
feats_f_trgt

'tst_feat_g19_w250.bp'

In [9]:
feats_curr = bp.unpack_ndarray_from_file(os.path.join('../input/feats_tblr', feats_f_curr))
feats_trgt = bp.unpack_ndarray_from_file(os.path.join('../input/feats_tblr', feats_f_trgt))

In [16]:
feats_diff = list(set(feats_trgt) - set(feats_curr))
feats_diff

['MA_5th_wndw_std_avg_R',
 'MA_10th_wndw_BBhigh_avg_R',
 'MA_10th_wndw_BBlow_avg_R',
 'MA_5th_wndw_std_avg_L',
 'MA_2nd_wndw_BBlow_avg_R',
 'MA_5th_wndw_BBhigh_avg_L',
 'binned_entropy_nonan_L',
 'MA_2nd_wndw_BBhigh_avg_R',
 'binned_entropy_nonan_3_L',
 'binned_entropy_nonan_2_L',
 'MA_5th_wndw_BBlow_avg_R',
 'MA_2nd_wndw_BBhigh_avg_L',
 'binned_entropy_nonan_4_L',
 'MA_10th_wndw_std_avg_L',
 'binned_entropy_nonan_R',
 'MA_2nd_wndw_BBlow_avg_L',
 'binned_entropy_nonan_4_R',
 'binned_entropy_nonan_2_R',
 'MA_10th_wndw_BBhigh_avg_L',
 'binned_entropy_nonan_1_L',
 'MA_2nd_wndw_std_avg_R',
 'binned_entropy_nonan_3_R',
 'MA_5th_wndw_BBhigh_avg_R',
 'MA_10th_wndw_std_avg_R',
 'binned_entropy_nonan_1_R',
 'MA_2nd_wndw_std_avg_L',
 'MA_10th_wndw_BBlow_avg_L',
 'MA_5th_wndw_BBlow_avg_L']

In [26]:
feats_fix = feats_diff
feats_fix_map = OrderedDict([(s, i) for i, s in zip(range(len(feats_fix)), feats_fix)])
feats_fix_map

OrderedDict([('MA_5th_wndw_std_avg_R', 0),
             ('MA_10th_wndw_BBhigh_avg_R', 1),
             ('MA_10th_wndw_BBlow_avg_R', 2),
             ('MA_5th_wndw_std_avg_L', 3),
             ('MA_2nd_wndw_BBlow_avg_R', 4),
             ('MA_5th_wndw_BBhigh_avg_L', 5),
             ('binned_entropy_nonan_L', 6),
             ('MA_2nd_wndw_BBhigh_avg_R', 7),
             ('binned_entropy_nonan_3_L', 8),
             ('binned_entropy_nonan_2_L', 9),
             ('MA_5th_wndw_BBlow_avg_R', 10),
             ('MA_2nd_wndw_BBhigh_avg_L', 11),
             ('binned_entropy_nonan_4_L', 12),
             ('MA_10th_wndw_std_avg_L', 13),
             ('binned_entropy_nonan_R', 14),
             ('MA_2nd_wndw_BBlow_avg_L', 15),
             ('binned_entropy_nonan_4_R', 16),
             ('binned_entropy_nonan_2_R', 17),
             ('MA_10th_wndw_BBhigh_avg_L', 18),
             ('binned_entropy_nonan_1_L', 19),
             ('MA_2nd_wndw_std_avg_R', 20),
             ('binned_entropy_nonan_3_R

In [27]:
feats_trgt_map = OrderedDict(sorted([(feats_fix_map[s], np.where(feats_trgt==s)[0].item()) for s in feats_diff], key=lambda tpl: tpl[1]))
feats_trgt_map

OrderedDict([(3, 56),
             (5, 57),
             (27, 58),
             (25, 59),
             (11, 60),
             (15, 61),
             (13, 62),
             (18, 63),
             (26, 64),
             (6, 312),
             (19, 313),
             (9, 314),
             (8, 315),
             (12, 316),
             (0, 402),
             (22, 403),
             (10, 404),
             (20, 405),
             (7, 406),
             (4, 407),
             (23, 408),
             (1, 409),
             (2, 410),
             (14, 658),
             (24, 659),
             (17, 660),
             (21, 661),
             (16, 662)])

In [29]:
array_curr = np.random.randint(0, 10, size=(10, 1000))
array_fix = np.stack([np.arange(i, i+10) for i in range(28)], -1)

In [32]:
for i0, i1 in feats_trgt_map.items():
    array_curr = np.insert(array_curr, i1, array_fix[:, i0], axis=1)

In [35]:
array_curr[:, [56, 57, 58, 59, 60, 312]]

array([[ 3,  5, 27, 25, 11,  6],
       [ 4,  6, 28, 26, 12,  7],
       [ 5,  7, 29, 27, 13,  8],
       [ 6,  8, 30, 28, 14,  9],
       [ 7,  9, 31, 29, 15, 10],
       [ 8, 10, 32, 30, 16, 11],
       [ 9, 11, 33, 31, 17, 12],
       [10, 12, 34, 32, 18, 13],
       [11, 13, 35, 33, 19, 14],
       [12, 14, 36, 34, 20, 15]])